In [1]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from pprint import pprint
import csv
import itertools as it
from prettytable import PrettyTable
# from nanrms import *
# from handover import *
# from mask import *
import matplotlib.pylab as plt
import seaborn as sns

pdir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # for jupyter-notebook
sys.path.insert(1, pdir)
from myutils import *

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度
# plt.style.use('ggplot')

pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

def set_data(df):
    df['seq'] = df['seq'].astype('Int32')
    # df['rpkg'] = df['rpkg'].astype('Int8')
    df['frame_id'] = df['frame_id'].astype('Int32')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['xmit_time'] = pd.to_datetime(df['xmit_time'])
    df['arr_time'] = pd.to_datetime(df['arr_time'])
    df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
    df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
    df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
    df['lost'] = df['lost'].astype('boolean')
    df['excl'] = df['excl'].astype('boolean')
    df['latency'] = df['latency'].astype('float32')
    return df

datadir = "/Users/jackbedford/Desktop/MOXA/Code/data"
path = "./temp"
exps = {
    "2023-05-07/_Bandlock_8_Schemes_Phone": ["#{:02d}".format(i+1) for i in range(7)],
}
title = '8_Schemes'
_devices = [
    ["sm00", "sm01", "sm02", "sm03", "sm04", "sm05", "sm06", "sm07",],
]
_schemes = [
    ["All", "B3", "B7", "B8", "B3B7", "B3B8", "B7B8", "LTE",],
]

In [3]:
i = -1
dfs_ul = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ul.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_ul[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_ul[i][j] = dfs_ul[i][j][(dfs_ul[i][j]['seq'] >= st) & (dfs_ul[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ul))

i = -1
dfs_dl = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_dl.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_dl[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_dl[i][j] = dfs_dl[i][j][(dfs_dl[i][j]['seq'] >= st) & (dfs_dl[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_dl))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm00/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm01/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm02/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm03/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm04/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm05/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm06/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm07/#01/data/udp_uplk_loss_latency.csv True
/Users/j

In [2]:
i = -1
dfs_ho = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ho.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            dbpath = os.path.join(datadir, exp, dev, trace, 'data')
            data = os.path.join(dbpath, [s for s in os.listdir(dbpath) if s.startswith('diag_log') and s.endswith('_rrc.csv')][0])
            # data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df, _, _ = mi_parse_ho(df, tz=8)
            df['ho_type0'] = df['ho_type'].astype('string')
            df.loc[df['cause'].notna(), 'ho_type0'] = df['ho_type'].astype('string') + '_' + df['cause'].astype('string')
            df['ho_type0'] = df['ho_type0'].astype('category')
            dfs_ho[i].append(df.copy())
        #     st.append(df['seq'].array[0])
        #     et.append(df['seq'].array[-1])
        # st, et = max(st), min(et)
        # for j, (dev, schm) in enumerate(zip(devices, schemes)):
        #     dfs_ho[i][j] = dfs_ho[i][j][(dfs_ho[i][j]['seq'] >= st) & (dfs_ho[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ho))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-07/_Bandlock_8_Schemes_Phone/sm00/#01/data/diag_log_sm00_2023-05-07_12-42-00_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-07/_Bandlock_8_Schemes_Phone/sm01/#01/data/diag_log_sm01_2023-05-07_12-42-00_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-07/_Bandlock_8_Schemes_Phone/sm02/#01/data/diag_log_sm02_2023-05-07_12-42-00_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-07/_Bandlock_8_Schemes_Phone/sm03/#01/data/diag_log_sm03_2023-05-07_12-42-00_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-07/_Bandlock_8_Schemes_Phone/sm04/#01/data/diag_log_sm04_2023-05-07_12-42-00_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-07/_Bandlock_8_Schemes_Phone/sm05/#01/data/diag_log_sm05_2023-05-07_12-42-00_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-07/_Bandlock_8_Schemes_Phone/sm06/#01/data/diag_log_sm06_2023-05-07_12-42-00_rrc.csv True
/Users/jackbedford/D

In [7]:
dfs_ho[3][0]

,start,end,ho_type,intr,sPCI,sFreq,tPCI,tFreq,snrPCI,tnrPCI,cause,others,st_scel,ePCI,ECI,eNB,BID,DL_Freq,DL_BW,UL_Freq,UL_BW,ePCI1,ECI1,eNB1,BID1,DL_Freq1,DL_BW1,UL_Freq1,UL_BW1,nrPCI,nrFreq,nrPCI1,nrFreq1,Timestamp,type_id,ho_type0
0,2023-05-07 13:16:12.660922,2023-05-07 13:16:12.677717,SN_HO,0.016795,378,3050,<NA>,<NA>,350,362,NaN,<NA>,0,378,<NA>,<NA>,<NA>,3050,NaN,<NA>,NaN,378,<NA>,<NA>,<NA>,3050,NaN,<NA>,NaN,350,631000,362,631000,2023-05-07 13:16:12.660922,RRC_OTA_Handover_Parsing,SN_HO
1,2023-05-07 13:16:18.406478,2023-05-07 13:16:18.422705,MN_HO,0.016227,378,3050,362,3050,362,<NA>,NaN,Set up 1 SCell.,1,378,<NA>,<NA>,<NA>,3050,NaN,<NA>,NaN,362,54512408,212939,7,3050,20 MHz,21050,20 MHz,362,631000,362,631000,2023-05-07 13:16:18.406478,RRC_OTA_Handover_Parsing,MN_HO
2,2023-05-07 13:16:52.550489,2023-05-07 13:16:52.567538,MN_HO,0.017049,362,3050,73,3050,362,<NA>,NaN,Set up 1 SCell.,1,362,54512408,212939,7,3050,20 MHz,21050,20 MHz,73,55671576,217467,7,3050,20 MHz,21050,20 MHz,362,631000,362,631000,2023-05-07 13:16:52.550489,RRC_OTA_Handover_Parsing,MN_HO
3,2023-05-07 13:16:53.323884,2023-05-07 13:16:53.340375,SN_HO,0.016491,73,3050,<NA>,<NA>,362,73,NaN,<NA>,0,73,55671576,217467,7,3050,20 MHz,21050,20 MHz,73,55671576,217467,7,3050,20 MHz,21050,20 MHz,362,631000,73,631000,2023-05-07 13:16:53.323884,RRC_OTA_Handover_Parsing,SN_HO
4,2023-05-07 13:16:54.644629,2023-05-07 13:16:54.661342,SN_HO,0.016713,73,3050,<NA>,<NA>,73,370,NaN,<NA>,0,73,55671576,217467,7,3050,20 MHz,21050,20 MHz,73,55671576,217467,7,3050,20 MHz,21050,20 MHz,73,631000,370,631000,2023-05-07 13:16:54.644629,RRC_OTA_Handover_Parsing,SN_HO
5,2023-05-07 13:16:54.822611,2023-05-07 13:16:54.839363,MN_HO,0.016752,73,3050,370,3050,370,<NA>,NaN,Set up 1 SCell.,1,73,55671576,217467,7,3050,20 MHz,21050,20 MHz,370,54512418,212939,7,3050,20 MHz,21050,20 MHz,370,631000,370,631000,2023-05-07 13:16:54.822611,RRC_OTA_Handover_Parsing,MN_HO
6,2023-05-07 13:17:00.879167,2023-05-07 13:17:00.958270,MCG_Failure,0.079103,73,1400,370,<NA>,370,<NA>,otherFailure (2),<NA>,0,73,55671574,217467,3,1400,10 MHz,19400,10 MHz,73,55671574,217467,3,1400,10 MHz,19400,10 MHz,370,631000,<NA>,<NA>,2023-05-07 13:17:00.879167,RRC_OTA_Handover_Parsing,MCG_Failure_otherFailure (2)
7,2023-05-07 13:17:01.861871,2023-05-07 13:17:01.890165,SN_Rel_MN_HO,0.028294,73,1400,73,3650,<NA>,<NA>,NaN,Set up 3 SCell. Inter-Freq HO. Near after RLF.,3,73,55671574,217467,3,1400,10 MHz,19400,10 MHz,73,55671573,217467,8,3650,10 MHz,21650,10 MHz,<NA>,<NA>,<NA>,<NA>,2023-05-07 13:17:01.861871,RRC_OTA_Handover_Parsing,SN_Rel_MN_HO
8,2023-05-07 13:17:02.009346,2023-05-07 13:17:02.030499,SN_Setup,0.021153,73,3650,<NA>,<NA>,<NA>,73,NaN,Set up 1 SCell.,1,73,55671573,217467,8,3650,10 MHz,21650,10 MHz,73,55671573,217467,8,3650,10 MHz,21650,10 MHz,<NA>,<NA>,73,631000,2023-05-07 13:17:02.009346,RRC_OTA_Handover_Parsing,SN_Setup
9,2023-05-07 13:17:06.536253,2023-05-07 13:17:06.552082,MN_HO,0.015829,73,3650,294,3650,73,<NA>,NaN,Set up 1 SCell.,1,73,55671573,217467,8,3650,10 MHz,21650,10 MHz,294,54687499,213623,8,3650,10 MHz,21650,10 MHz,73,631000,73,631000,2023-05-07 13:17:06.536253,RRC_OTA_Handover_Parsing,MN_HO


In [37]:
for i in range(len(dfs_ho)):
    for j in range(len(devices)):
        df = dfs_ho[i][j].copy()
        if sum(df['cause'] == 'rlc-MaxNumRetx (2)') > 0:
            print(f'sm{j}', f'trace#{i+1}', '∆')
        else:
            print(f'sm{j}', f'trace#{i+1}', 'O')

sm0 trace#1 O
sm1 trace#1 ∆
sm2 trace#1 ∆
sm3 trace#1 ∆
sm4 trace#1 ∆
sm5 trace#1 ∆
sm6 trace#1 ∆
sm7 trace#1 O
sm0 trace#2 ∆
sm1 trace#2 ∆
sm2 trace#2 O
sm3 trace#2 O
sm4 trace#2 ∆
sm5 trace#2 ∆
sm6 trace#2 ∆
sm7 trace#2 O
sm0 trace#3 O
sm1 trace#3 ∆
sm2 trace#3 ∆
sm3 trace#3 ∆
sm4 trace#3 ∆
sm5 trace#3 ∆
sm6 trace#3 ∆
sm7 trace#3 O
sm0 trace#4 ∆
sm1 trace#4 O
sm2 trace#4 O
sm3 trace#4 O
sm4 trace#4 O
sm5 trace#4 O
sm6 trace#4 O
sm7 trace#4 O
sm0 trace#5 O
sm1 trace#5 ∆
sm2 trace#5 ∆
sm3 trace#5 ∆
sm4 trace#5 ∆
sm5 trace#5 O
sm6 trace#5 O
sm7 trace#5 O
sm0 trace#6 ∆
sm1 trace#6 ∆
sm2 trace#6 ∆
sm3 trace#6 ∆
sm4 trace#6 ∆
sm5 trace#6 ∆
sm6 trace#6 ∆
sm7 trace#6 O
sm0 trace#7 O
sm1 trace#7 O
sm2 trace#7 O
sm3 trace#7 O
sm4 trace#7 O
sm5 trace#7 O
sm6 trace#7 O
sm7 trace#7 O
